In [48]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd

Урок #2

In [24]:
url = "https://health-diet.ru/table_calorie/"

headers = {
    "accept": "*/*",
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
}
   
req = requests.get(url, headers = headers)

src = req.text
#print(src)

#Сохранение страницы
with open("food.html", "w", encoding="utf-8-sig") as file:
    file.write(src)

with open("food.html", encoding="utf-8-sig") as file:
     src = file.read()
soup = BeautifulSoup(src, "lxml")

#Получение ссылок
all_products_hrefs = soup.find_all(class_="mzr-tc-group-item-href")

#Запись в файл json
import json 
all_categories_dict = {}
for item in all_products_hrefs:
    item_text = item.text
    item_href = "https://health-diet.ru" + item.get("href")
    all_categories_dict[item_text] = item_href

with open("all_categories_dict.json", "w") as file:
    json.dump(all_categories_dict, file, indent = 4, ensure_ascii = False)
    
with open("all_categories_dict.json") as file:
    all_categories = json.load(file)
    
iteration_count = int(len(all_categories)) - 1
count = 0
print(f"Всего итераций:{iteration_count}")
#Замена символов в нескольих элементах + Цикл сбора информации из каждого раздела о продукте и его хим составе, записывая в файл
for category_name, category_href in all_categories.items():
    rep = [",", " ", "-", "'"]
    for item in rep:
        if item in category_name:
            category_name = category_name.replace(item, "_")
    req = requests.get(url=category_href, headers=headers)
    src = req.text
    with open(f"data/{count}_{category_name}.html", "w", encoding="utf-8-sig") as file:
        file.write(src)
    with open(f"data/{count}_{category_name}.html", encoding="utf-8-sig") as file:
        src = file.read()
    soup = BeautifulSoup(src, "lxml")
    
#Проверка страницы на наличие таблицы с продуктами
    alert_block = soup.find(class_="uk-alert-danger")
    if alert_block is not None: 
        continue
        
#Сбор заголовков таблицы

    table_head = soup.find(class_="mzr-tc-group-table").find("tr").find_all("th")
        
    product = table_head[0].text
    calories = table_head[1].text
    proteins = table_head[2].text
    fats = table_head[3].text
    carbohydrates = table_head[4].text
        
    with open(f"data/{count}_{category_name}.csv", "w", encoding="utf-8-sig", newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(
            (
                product,
                calories,
                proteins,
                fats,
                carbohydrates
            )
        )
        
#Сбор информации из тегов tr
    products_data = soup.select('.mzr-tc-group-table tbody tr')
    product_info = []

    for item in products_data:
        product_tds = item.find_all("td")
            
        title = product_tds[0].find('a').text
        calories = product_tds[1].text
        proteins = product_tds[2].text
        fats = product_tds[3].text
        carbohydrates = product_tds[4].text
            #print(title)
        product_info.append(
                {
                    "Title": title,
                    "Calories": calories,
                    "Proteins": proteins,
                    "Fats": fats,
                    "Carbohydrates": carbohydrates
                }
            )
        
        with open(f"data/{count}_{category_name}.csv", "a",  encoding="utf-8", newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow( 
                (
                    title,
                    calories,
                    proteins,
                    fats,
                    carbohydrates
                )
            )
    with open(f"data/{count}_{category_name}.json", "a",  encoding="utf-8-sig") as file:
                json.dump(product_info, file, indent = 4, ensure_ascii = False)
    count += 1
    

Всего итераций:53


Парсинг таблицы

In [62]:
import glob

In [67]:
#Выбор файлов для объединения
files = glob.glob("data/*.csv")
files

['data\\0_Баранина_и_дичь.csv',
 'data\\10_Напитки_алкогольные.csv',
 'data\\11_Овощи_и_зелень.csv',
 'data\\12_Орехи_и_семена.csv',
 'data\\13_Полуфабрикаты.csv',
 'data\\14_Птица.csv',
 'data\\15_Ресторанная_еда.csv',
 'data\\16_Рыба_и_морепродукты.csv',
 'data\\17_Свинина.csv',
 'data\\18_Снеки.csv',
 'data\\19_Соки.csv',
 'data\\1_Бобовые.csv',
 'data\\20_Сосиски_и_колбаса.csv',
 'data\\21_Травы__специи_и_соусы.csv',
 'data\\22_Фаст_фуд.csv',
 'data\\23_Фрукты_и_ягоды.csv',
 'data\\24_Хлеб_и_выпечка.csv',
 'data\\25_Яйца_и_продукты_из_яиц.csv',
 'data\\26_Варенье_и_джемы.csv',
 'data\\27_Вторые_блюда.csv',
 'data\\28_Выпечка.csv',
 'data\\29_Гарниры.csv',
 'data\\2_Вода_и_напитки.csv',
 'data\\30_Десерты.csv',
 'data\\31_Заготовки.csv',
 'data\\32_Закуски.csv',
 'data\\33_Каши.csv',
 'data\\34_Напитки.csv',
 'data\\35_Первые_блюда.csv',
 'data\\36_Салаты.csv',
 'data\\37_Соусы_и_заправки.csv',
 'data\\38_Campina.csv',
 'data\\39_Coca_Cola.csv',
 'data\\3_Говядина_и_телятина.csv',
 

In [75]:
#Процесс объединения
combined = pd.DataFrame()

for file in files:
    data = pd.read_csv(file, sep=";")
    data['Список'] = file
    combined = pd.concat([combined, data])

In [73]:
#Проверка результата
combined[:5]

,Продукт,Калорийность,Белки,Жиры,Углеводы,Общий список
0,Антилопа,114 кКал,"22,38 г","2,03 г",0 г,data\0_Баранина_и_дичь.csv
1,"Антилопа, запеченная",150 кКал,"29,45 г","2,67 г",0 г,data\0_Баранина_и_дичь.csv
2,Баранина 1 кат.,209 кКал,"15,6 г","16,3 г",0 г,data\0_Баранина_и_дичь.csv
3,Баранина 2 кат.,166 кКал,"19,8 г","9,6 г",0 г,data\0_Баранина_и_дичь.csv
4,"Баранина австралийская ,голяшка, мясо и жир, о...",231 кКал,"25,25 г","13,69 г",0 г,data\0_Баранина_и_дичь.csv


In [77]:
#Запись в отдельный csv
combined.to_csv('Общий список', index=False, sep=";")